In [1]:
import nltk
import pandas as pd
import re
import tensorflow as tf
import time
import multiprocessing
import io
import gensim
import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras
from sklearn.model_selection import KFold # import KFold
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Activation, Embedding, LSTM, Bidirectional, Dropout, GRU
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from datetime import timedelta
from gensim.models import word2vec
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from varname import nameof

from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#2.Load Data

In [7]:
#simpan path dataset
path_data_aspek = "data_scraaping_kotaTegal_Labeling.csv"
 
#read dataset
data_aspek = pd.read_csv(path_data_aspek, sep=";", header=[0], encoding="UTF-8")

#3.Implementasi Preprocessing

In [8]:
#menyimpan tweet. (tipe data series pandas)
data_content = data_aspek['tweet']

##3.1. Casefolding

In [9]:
# casefolding
data_casefolding = data_content.str.lower()
data_casefolding.head()

0    [tf] yg jual gini area slawi aja dimana ya ges...
1    [tf] spill game offline kalian yg menarik itu ...
2    [tf] cara hapus tag contact di get contact yan...
3    selamat bertugas kepada peserta bsi explore ya...
4    minggu, 15 januari 2023 petugas pleton c regu ...
Name: tweet, dtype: object

##3.2. Filtering

In [10]:
#filtering

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet) for tweet in data_casefolding]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]  #hapus simbol'[!#?,.:";@()-_/\']'
#  hapus #tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

# # filtering RT , @ dan #
# fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
# clean = [fungsi_clen_rt for tweet in filtering_numeric]

data_filtering = pd.Series(filtering_numeric)


## 3.3. Tokenisasi

In [11]:
# #tokenize
tknzr = TweetTokenizer()
data_tokenize = [tknzr.tokenize(tweet) for tweet in data_filtering]

##3.4. Konversi Slangword

In [ ]:
#slang word
path_dataslang = open("kamus kata baku-clear (1).csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

data_formal = []
for data in data_tokenize:
  data_clean = [replaceSlang(word) for word in data]
  data_formal.append(data_clean)
len_data_formal = len(data_formal)
# print(data_formal)
# len_data_formal

In [16]:
dataslang

,0,1
0,tlg,tolong
1,dwpan,depan
2,dwngan,dengan
3,byk,banyak
4,smoga,semoga
...,...,...
5437,politic,politik
5438,cont,
5439,rt,
5440,cc,NaN


## 3.5. Stopword

In [17]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:  
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
reviews = [removeStopWords(line,stopwords) for line in data_formal]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
type(reviews)

list

# 4.Implementasi Naive Bayes (Klasifikasi Aspek)

## 4.1.pembuatan vector kata

In [19]:
# pembuatan vector kata
vectorizer = TfidfVectorizer()
reviews2 = [" ".join(r) for r in reviews]
vektor_tfidf = vectorizer.fit_transform(reviews2)
vektor_tfidf = vektor_tfidf.toarray()
vektor_tfidf.shape

(1012, 3852)

In [21]:
print(vektor_tfidf)
# print(vectorizer.get_feature_names())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## 4.2.split data dan cross-validation

In [22]:
data_aspek.head()

,date,username,tweet,wisata/hiburan,pendidikan,fasilitas/layanan publik,kuliner
0,16/01/2023 10:08,tegalfess,[tf] yg jual gini area slawi aja dimana ya ges...,-1,-1,-1,-1
1,16/01/2023 09:56,tegalfess,[tf] spill game offline kalian yg menarik itu ...,1,-1,-1,-1
2,16/01/2023 09:47,tegalfess,[tf] cara hapus tag contact di get contact yan...,-1,-1,-1,-1
3,16/01/2023 09:41,BSI_Tegal,Selamat Bertugas kepada peserta BSI Explore ya...,-1,1,-1,-1
4,16/01/2023 09:39,damkarbekasi,"Minggu, 15 Januari 2023 petugas Pleton C Regu ...",-1,-1,1,-1


## Aspek Wisata

In [ ]:
#Panggil Data target aspek (y)
label_aspek_1 = data_aspek['wisata/hiburan']

## Aspek Pendidikan

In [ ]:
label_aspek_2 = data_aspek['pendidikan']

## Aspek Fasilitas dan Layanan Public

In [ ]:
label_aspek_3 = data_aspek['fasilitas/layanan publik']

## Aspek Kuliner

In [23]:
label_aspek_4 = data_aspek['kuliner']

In [55]:
X_train, X_test, y_train, y_test = train_test_split(vektor_tfidf, label_aspek_4, test_size=0.2)#, shuffle=True)
nama = "Model_Aspek_Kuliner"
y_train = np.array(y_train)
y_test = np.array(y_test)

In [56]:
print(y_test)
p = np.where(y_test==1)
np.array(p).shape

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1  1
  1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1  1 -1 -1 -1  1  1 -1  1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  1 -1 -1 -1  1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1  1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1  1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1  1 -1 -1
 -1 -1 -1 -1  1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


(1, 24)

In [44]:
# Cross-validasi
kf = StratifiedKFold(n_splits=2)
kf.get_n_splits(X_train) 
print(kf)

StratifiedKFold(n_splits=2, random_state=None, shuffle=False)


# 5.Training Naive Bayes

In [45]:
# fungsi confusion matrix
def hitung_score_nvb(y_test, y_pred):
    tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred).ravel()
    print(" True Positif : " + str(tp),
          "True Negatif : " + str(tn), 
          "False Positif : " + str(fp), 
          "False Negatif : " + str(fn) + '\n')
    # print(fp, tp)
    acc_nvb = (tp+tn) / (tp + fp + tn + fn)
    prec_nvb = tp / (tp + fp)
    recc_nvb = tp / (tp + fn)
    f_1_nvb = 2 * (prec_nvb * recc_nvb) / (prec_nvb + recc_nvb)
    return acc_nvb, prec_nvb, recc_nvb, f_1_nvb

In [46]:
from sklearn.utils.multiclass import unique_labels
def plot(test_y, pred_y):
  labels = unique_labels(y_test)
  column = [f'Prediksi {label}' for label in labels]
  indeces = [f'Actual {label}' for label in labels]
  table = pd.DataFrame(confusion_matrix(test_y, pred_y),
                       columns = column, index = indeces)
  return table

In [47]:
# print(plot(label_asli, pred_y_nvb))

In [48]:
kf_nvb_number = 0
akurasi_nvb_arr = []
prec_nvb_arr = []
rec_nvb_arr = []
f1_nvb_arr = []
score = {
    'Akurasi':[],
    'Presisi':[],
    'Recall':[],
    'F1':[]
}
index_fold = 1

for train_aspek_nvb, test_aspek_nvb in kf.split(X_train, y_train):
  X_train_aspek_nvb, X_test_aspek_nvb = X_train[train_aspek_nvb], X_train[test_aspek_nvb]
  y_train_aspek_nvb, y_test_aspek_nvb = y_train[train_aspek_nvb], y_train[test_aspek_nvb]
 
  kf_nvb_number += 1
 
  # train model
  clf = MultinomialNB()
  clf.fit(X_train_aspek_nvb, y_train_aspek_nvb)
 
  # save model
  # model_nvb = 'tfidf_sentimen_integritas_nvb.sav'
  pickle.dump(clf, open('Model/tfidf_{}_nvb.pkl'.format(nama), 'wb'))
 
  # pengujian
  pred_y_nvb = clf.predict(X_test_aspek_nvb)
  label_asli = y_test_aspek_nvb
  
  print('\n'+'-'*45)
  print('confusion matrix : ' + str(nama)+'\n' + 'Fold ke-' + str(kf_nvb_number))
  print('-'*45,'\n')

  # print('confusion matrix :')
  # print('-'*27,'\n')
  print(plot(label_asli, pred_y_nvb),'\n')

  # print(confusion_matrix(label_asli, pred_y_nvb))
  acc, prec, recc, f_1 = hitung_score_nvb(label_asli, pred_y_nvb)

  # print(prediction[:9])
  # print(y_test_aspek_nvb[:9])
 
  print("Score fold ke-" + str(kf_nvb_number),'\n')
  print("Naive Bayes Accuracy Score -> ", str(float("%.2f" % round(accuracy_score(pred_y_nvb, y_test_aspek_nvb)*100, 2))) + " %")
  print("Naive Bayes Precission Score -> ", str(float("%.2f" % round(precision_score(pred_y_nvb, y_test_aspek_nvb, average='macro')*100, 2)))+ " %")
  print("Naive Bayes Recall Score -> ", str(float("%.2f" % round(recall_score(pred_y_nvb, y_test_aspek_nvb, average='macro')*100, 2)))+ " %")
  print("Naive Bayes F1 Measurement Score -> ", str(float("%.2f" % round(f1_score(pred_y_nvb, y_test_aspek_nvb, average='macro')*100, 2)))+ " %" + "\n")
  
  # print("confusion matrix :")
  # print(confusion_matrix(label_asli, pred_y_nvb))
  # print("\n","True positif : " + str(cm[1][1]), "\n",
  #       "True negatif : " + str(cm[0][0]), "\n",
  #       "False positif : " + str(cm[0][1]), "\n",
  #       "False negatif : " + str(cm[1][0]))
  index_fold += 1

  score['Akurasi'].append(acc)
  score['Presisi'].append(prec)
  score['Recall'].append(recc)
  score['F1'].append(f_1)
  akurasi_nvb_arr.append(accuracy_score(pred_y_nvb, y_test_aspek_nvb))
  prec_nvb_arr.append(precision_score(pred_y_nvb, y_test_aspek_nvb,average='macro'))
  rec_nvb_arr.append(recall_score(pred_y_nvb, y_test_aspek_nvb,average='macro'))
  f1_nvb_arr.append(f1_score(pred_y_nvb, y_test_aspek_nvb,average='macro'))
  
  print('-'*100,'\n')
# from termcolor import colored
# print(colored("Rata-rata Akurasi NVB : " + str(round(np.mean(akurasi_nvb_arr)*100,2, 'red', attrs=['bold'])),"%")
print("Rata-rata Akurasi NVB : " + str(round(np.mean(akurasi_nvb_arr)*100,2)), "%")
print("Precision K-Fold : " + str(round(np.mean(prec_nvb_arr)*100,2)), "%")
print("Recall K-Fold : " + str(round(np.mean(rec_nvb_arr)*100,2)), "%")
print("F1 K-Fold : " + str(round(np.mean(f1_nvb_arr)*100,2)), "%")
print("Fold-1 : " +str(score['Akurasi'][0]) + "%", 
      "Fold-2 : " + str(score['Akurasi'][1]) + "%",
      "Fold-3 : " + str(score['Akurasi'][2]) + "%", 
      "Fold-4 : " + str(score['Akurasi'][3]) + "%") 
      #"Fold-5 : " + str(score['Akurasi'][4]) + "%")


---------------------------------------------
confusion matrix : Model_Aspek_Kuliner
Fold ke-1
--------------------------------------------- 

           Prediksi -1  Prediksi 0  Prediksi 1
Actual -1          323           0           0
Actual 0             1           0           0
Actual 1            30           0           0 



ValueError: too many values to unpack (expected 4)

# 6.Pengujian

In [ ]:
indeks=np.where(score["Akurasi"]==np.amax(score['Akurasi']))[0]

model = pickle.load(open('/content/drive/MyDrive/1 Rizki Folder/Hibah bersama Dosen/result_baru/nvb/tfidf_'+str(nama)+'_nvb.pkl','rb'))
pred_y_ = model.predict(X_test)
label_test = y_test
# result = model.score(X_test, y_test)
print('\n'+'-'*45)
print('confusion matrix : ' + str(nama))
print('-'*45,'\n')
print(plot(label_test, pred_y_),'\n')
# print("Confusion Matrix :")
# print('-'*20,'\n')

# print(confusion_matrix(label_test, pred_y_))
acc, prec, recc, f_1 = hitung_score_nvb(label_test, pred_y_)
from termcolor import colored
print(colored("Pengujian " +str(nama) + ': ',  'red', attrs=['bold']))
print('Model Terbaik Fold ke-' +str(indeks[0]+1, ),'\n',
      'Akurasi  : ' + str(round((acc*100) ,2)) + "%","\n",
      'Presisi  : '+ str(round((prec*100) ,2))+ "%","\n", 
      'Recall  : '+ str(round((recc*100) ,2))+ "%","\n", 
      'F1 Score  : '+ str(round((f_1*100) ,2))+ "%",'\n')


---------------------------------------------
confusion matrix : aspek_aksesbilitas_v3
--------------------------------------------- 

          Prediksi 0  Prediksi 1
Actual 0        1157           0
Actual 1         208           0 

 True Positif : 0 True Negatif : 1157 False Positif : 0 False Negatif : 208

Pengujian aspek_aksesbilitas_v3: 
Model Terbaik Fold ke-4 
 Akurasi  : 84.76% 
 Presisi  : nan% 
 Recall  : 0.0% 
 F1 Score  : nan% 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
f_1

nan

# Augmentasi data 
Bisa menggunakan Smote, adasyn, dan randomoversampler

In [59]:
# Contoh menggunakan Adasyn
from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = ADASYN().fit_resample(X_train, y_train)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 3, n_neighbors = 6

In [ ]:
# contoh menggunakan Smote
X_resampled, y_resampled = SMOTE().fit_resample(X, y)

NameError: ignored

In [ ]:
# contoh menggunakan RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(vektor_tfidf, label_aspek_atraksi)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[(0, 4354), (1, 4354)]


In [ ]:
clf = MultinomialNB()
clf.fit(X_resampled, y_resampled)
pred = clf.predict(X_resampled)
print(clf.score(X_resampled, y_resampled))
print(precision_score(pred, y_resampled))
print(recall_score(pred, y_resampled))
print(f1_score(pred, y_resampled))
print(X_resampled.argmax(axis=1))

0.9297197978870004
0.9127239320165366
0.9448407037565383
0.9285046728971963
[2573 2697  562 ... 3821 6071 1067]
